# Time series interpolating with aeon

Suppose we have a set of time series with different lengths, i.e. different number
of time points. Currently, most of aeon's functionality requires equal-length time series, so to use aeon, we need to first converted our data into equal-length time series. In this tutorial, you will learn how to use the `TSInterpolator` to do so.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from aeon.classification.convolution_based import RocketClassifier
from aeon.datasets import load_basic_motions

C:\Code\aeon\aeon\base\__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
C:\Code\aeon\venv\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


ModuleNotFoundError: No module named 'aeon.transformations.compose'

## Ordinary situation

Here is a normal situation, when all time series have same length. We load an example
 data set from aeon and train a classifier.

In [ ]:
X, y = load_basic_motions()
X_train, X_test, y_train, y_test = train_test_split(X, y)
clf = RocketClassifier(num_kernels=200)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

## If time series are unequal length, aeon's algorithm may raise an error

Ifwe have unequal-length time series, classifiers without the capability to handle
this will raise an error.

In [ ]:
from aeon.datasets import load_plaid

X, y = load_plaid()
X_train, X_test, y_train, y_test = train_test_split(X, y)

try:
    clf = RocketClassifier(num_kernels=200)
    clf.fit(X_train, y_train)
    clf.score(X_test, y_test)
except ValueError as e:
    print(f"ValueError: {e}")

# Now the interpolator enters
Now we use our interpolator to resize time series of different lengths to user-defined length. Internally, it uses linear interpolation from scipy and draws equidistant samples on the user-defined number of points.

After interpolating the data, the classifier works again.

In [5]:
from aeon.transformations.collection.interpolate import TSInterpolator

steps = [
    ("transform", TSInterpolator(50)),
    ("classify", RocketClassifier()),
]
clf = Pipeline(steps)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.7732342007434945